In [ ]:
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        100,            #  3rd last digit before decimal
        10,             #  2nd last
        1,              #  last
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

# ---

m, c = var('m, c', positive=True)

u2, u3, u4= var("u2, u3, u4")
F1, F2, F3, F4, F5 = var("F1, F2, F3, F4, F5")
u = Matrix([0, u2, u3, u4, 0])
f = Matrix([F1, F2, F3, F4, F5])

# Statics with full 5x5-matrix to fin
# displacements and support reactions:
K = Matrix([
[  2*c,  -2*c,     0,      0,      0],
[ -2*c,   3*c,    -c,      0,      0],
[    0,    -c,   2*c,     -c,      0],
[    0,     0,    -c,    3*c,   -2*c],
[    0,     0,     0,   -2*c,    2*c],
])

eq = Eq(K*u, f)
sol = solve(eq,[u2, u3, u4, F1, F5])
pprint(sol)

# Statics with 3x3-matrix to find
# displacements only:
u = Matrix([u2, u3, u4])
f = Matrix([F2, F3, F4])
K = Matrix([
[3*c,  -c,   0],
[ -c, 2*c,  -c],
[  0,  -c, 3*c],
])
eq = Eq(K*u, f)
sol = solve(eq,[u2, u3, u4, F1, F5])
pprint(sol)


M = Matrix([
[m, 0, 0],
[0, m, 0],
[0, 0, m],
])

# ξ = λ²
xi = var("xi")
A = K + xi*M

pprint("\nCharacteristic equation:")
eq = Eq(det(A))
pprint(eq)

sol_xi = solve(eq,[xi])

u2, u3, u4 = var("u2, u3, u4")
u = Matrix([u2, u3, u4])

zero = Matrix([0,0,0])
for i in range(len(sol_xi)):
    pprint("\n\nEigenvalue:")
    xii = sol_xi[i]
    pprint(xii)
    Ai = A.subs(xi,xii)
    eq = Eq(Ai*u,zero)
    sol = solve(eq, [u2, u3])
    pprint("\nEigenvector:")
    pprint(sol)

# ⎧      5⋅F₂   F₃   F₄        F₂   F₃   5⋅F₄      5⋅F₂ + 3⋅F₃ + F₄      F₂ + 3⋅F₃ + F₄      F₂ + 3⋅F₃ + 5⋅F₄⎫
# ⎨F₁: - ──── - ── - ──, F₅: - ── - ── - ────, u₂: ────────────────, u₃: ──────────────, u₄: ────────────────⎬
# ⎩       6     2    6         6    2     6              12⋅c                 4⋅c                  12⋅c      ⎭
# ⎧    5⋅F₂ + 3⋅F₃ + F₄      F₂ + 3⋅F₃ + F₄      F₂ + 3⋅F₃ + 5⋅F₄⎫
# ⎨u₂: ────────────────, u₃: ──────────────, u₄: ────────────────⎬
# ⎩          12⋅c                 4⋅c                  12⋅c      ⎭
#
# Characteristic equation:
#      2                                      2
# - 2⋅c ⋅(3⋅c + m⋅ξ) + (2⋅c + m⋅ξ)⋅(3⋅c + m⋅ξ)  = 0
#
#
# Eigenvalue:
# -4⋅c
# ─────
#   m
#
# Eigenvector:
# {u₂: u₄, u₃: -u₄}
#
#
# Eigenvalue:
# -3⋅c
# ─────
#   m
#
# Eigenvector:
# {u₂: -u₄, u₃: 0}
#
#
# Eigenvalue:
# -c
# ───
#  m
#
# Eigenvector:
# {u₂: u₄, u₃: 2⋅u₄}
